In [1]:
import os
import importlib
import random
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

from interface import Environment
import agent.fix_rule.agent as aa
import agent.xuance.agent as xu

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
agent1 = aa.Agent()
agent2 = aa.Agent()

In [3]:
map_path = 'maps/1000_1000_2_10_vs_2_10.map'
# map_path = 'maps/1000_1000_fighter10v10.map'

In [4]:
env = Environment(map_path=map_path, side1_obs_ind=agent1.get_obs_ind(), side2_obs_ind=agent2.get_obs_ind())
step_cnt = 0

In [5]:
size_x, size_y = env.get_map_size()
side1_detector_num, side1_fighter_num, side2_detector_num, side2_fighter_num = env.get_unit_num()

In [6]:
agent1.set_map_info(size_x, size_y, side1_detector_num, side1_fighter_num)
agent2.set_map_info(size_x, size_y, side2_detector_num, side2_fighter_num)

In [9]:
side1_obs_dict, side2_obs_dict = env.get_obs()

In [10]:
for i in range(135):
    step_cnt += 1
    side1_detector_action, side1_fighter_action = agent1.get_action(side1_obs_dict, step_cnt)
    side2_detector_action, side2_fighter_action = agent2.get_action(side2_obs_dict, step_cnt)
    a=env.step(side1_detector_action, side1_fighter_action, side2_detector_action, side2_fighter_action)
    side1_obs_dict, side2_obs_dict = env.get_obs()

In [11]:
side1_obs_dict['joint_obs_dict']

{'strike_list': [], 'passive_detection_enemy_list': []}

In [14]:
side1_obs_dict['detector_obs_list']

[{'id': 1,
  'alive': True,
  'pos_x': 235,
  'pos_y': 334,
  'course': 0,
  'r_iswork': True,
  'r_fre_point': 1,
  'r_visible_list': [{'id': 4, 'type': 1, 'pos_x': 504, 'pos_y': 263},
   {'id': 5, 'type': 1, 'pos_x': 500, 'pos_y': 285}],
  'last_reward': 0,
  'last_action': {'course': 0, 'r_iswork': True, 'r_fre_point': 1}},
 {'id': 2,
  'alive': True,
  'pos_x': 235,
  'pos_y': 667,
  'course': 0,
  'r_iswork': True,
  'r_fre_point': 1,
  'r_visible_list': [],
  'last_reward': 0,
  'last_action': {'course': 0, 'r_iswork': True, 'r_fre_point': 1}}]

In [ ]:
a=env.step(side1_detector_action, side1_fighter_action, side2_detector_action, side2_fighter_action)

In [ ]:
side1_obs_dict['fighter_obs_list'][0]

In [ ]:
env.get_reward()

In [ ]:
env.get_map_size()[0]

In [ ]:
side2_obs_dict['detector_obs_list']

In [ ]:
side1_obs_dict['detector_obs_list']

In [ ]:
side1_obs_dict['fighter_obs_list']

In [ ]:
side1_obs_dict['joint_obs_dict']

In [ ]:
side2_obs_dict['fighter_obs_list']

In [ ]:
env.get_reward()

In [ ]:
ae = agent.ae.ae.StateAE()

In [ ]:
a=torch.Tensor([[1,2],[3,4]])
b=torch.Tensor([[0,0],[0,0]])

In [ ]:
a=torch.Tensor([1,2])
b=torch.Tensor([3,4])

In [ ]:
a=Variable(a, requires_grad=True)
b=Variable(b, requires_grad=False)

In [ ]:
class A(nn.Module):
    def __init__(self):
        super(A, self).__init__()
        self.linear = nn.Linear(2,2)
    
    def forward(self, input):
        return self.linear(input)

In [ ]:
net = A()
opt = optim.SGD(net.parameters(), lr=1)

In [ ]:
a=np.zeros((2,1,2))
b=np.ones((2,1,2))

In [ ]:
a=np.array([[1,2],[4,5],[3,4]])

In [ ]:
class BlockNet(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, out_activation=nn.ReLU):
        super(BlockNet, self).__init__()
        self.res1 = ResUnit(in_channels, in_channels, kernel_size=kernel_size)
        self.res2 = ResUnit(in_channels, out_channels, kernel_size=kernel_size, out_activation=out_activation)

    def forward(self, x):
        out = self.res1(x)
        out = self.res2(out)
        return out

In [ ]:
class ResUnit(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, out_activation=nn.ReLU):
        super(ResUnit, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size, 1, (kernel_size-1)//2)
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size, 1, (kernel_size-1)//2)
        if (in_channels != out_channels):
            self.projection = nn.Conv2d(
                in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        if out_activation:
            self.out_activation = out_activation()

    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.conv2(out)
        if (x.shape[1] != out.shape[1]):
            x = self.projection(x)
        out = out + x
        if hasattr(self, 'out_activation'):
            out = self.out_activation(out)
        return out

In [ ]:
        preprocess_net = nn.Sequential( # 预处理
            BlockNet(in_channels=2, out_channels=2, kernel_size=7, out_activation=nn.Tanh),
            BlockNet(in_channels=2, out_channels=2, kernel_size=5, out_activation=nn.Tanh))

In [ ]:
class DQN(nn.Module):
    def __init__(self, preprocess_net=None):
        super(DQN, self).__init__()
        self.tau = 0.005
        self.preprocess = preprocess_net
        self.q = BlockNet(in_channels=3, out_channels=1, kernel_size=3, out_activation=None)
        self.target_q = BlockNet(in_channels=3, out_channels=1, kernel_size=3, out_activation=None)
        self.opt = optim.SGD(self.parameters(), lr=1e-3)

In [ ]:
dqn = DQN(preprocess_net=preprocess_net)

In [13]:
a=np.array([[1,2],[3,4]])

In [16]:
[x for x in a if x[0]==1]

[array([1, 2])]

In [12]:
np.sum(a)

2